In [4]:
require 'poisson'

class Modeling
  def initialize(people_num, possibility_correct, lambda_poisson)
    @people_num = people_num
    @people_half = half_num(@people_num) # 終了の値や半分の値に用いる
    @possibility_correct = possibility_correct
    @lambda_poisson = lambda_poisson # ポアソン分布の平均lambda
    @collecting_deadline = lambda_poisson * 2 # 意見収集の締め切り時刻を平均の二倍とする
    @poisson = Poisson.new(@lambda_poisson) # ポアソン分布を扱うクラスのインスタンス
  end

  WEIGHT = 0.5.to_f

  # 人がいつ来るか配列で表す EXAMPLE [1, 0, 1] => 時刻0と時刻2に人がくる
  def simulate_when_people_come(people_num)
    when_people_come = Array.new(@collecting_deadline)
    people_num.times do |t|
      when_people_come[t] = 1
    end
    when_people_come.shuffle
  end


  # 一人目の意見を採用するというアルゴリズム
  def baseline_method_deciding_by_first_person_with_poisson(possibility_correct)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - 重み * かかる時間(決定する時刻 / collecting_deadline)
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ないときは飛ばす
        method_utility += possibility_correct - WEIGHT * (index.to_f / @collecting_deadline)
        break # 一人目の意見しか必要ないのですぐBREAK
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 引数majority_vote_people人で多数決して意見集約を行うというアルゴリズム
  def baseline_method_deciding_by_majority_vote_with_poisson(possibility_correct, majority_vote_people)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - 重み* かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      next if t_people < majority_vote_people # majority_vote_people人集まらない場合は、utility = 0
      when_people_come.shuffle.each_with_index do |elem, index|
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count += 1
        if people_count == majority_vote_people
          method_utility +=  (1 - relative_error_by_majority_vote(people_count, possibility_correct)) - WEIGHT * (index.to_f / @collecting_deadline)
          break
        end
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 先にhalf_num(引数temp_people_num)人集まった方の意見を採用するアルゴリズム
  def baseline_method_deciding_by_half_opinion_with_poisson(possibility_correct, temp_people_num)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - 重み * かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      people_count_arr = [] # EXAMPLE [4, 9, 20, 40, 45] 一人目は時刻4にきた　二人目は時刻9にきた...
      next if t_people < temp_people_num # temp_people_num人集まらない場合は、utility = 0 TODO temp_people_num人あつまらなくてもその半数は集まることはあるので厳密ではない
      when_people_come.each_with_index do |elem, index|
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count_arr << index
        people_count += 1
        if people_count == temp_people_num
          relative_error_array = relative_error_array_by_half_opinion(half_num(people_count), possibility_correct)
          expected_error = relative_error_array.inject(:+)
          average_index = 0
          relative_error_array.each_with_index do |e, i| # もっと簡単にかけそう
            average_index += people_count_arr[i + 2] * (e / expected_error)
          end
          method_utility =  (1 - expected_error) - WEIGHT * (average_index.to_f / @collecting_deadline)
          break
        end
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  # 誰か一人集まりかつ、時刻引数time_limitで意見収集を打ち切り,多数決で意思決定を行う
  def baseline_method_deciding_by_time_limit_with_poisson(possibility_correct, time_limit)
    average_method_utility = 0
    # 0~@collecting_deadline人来るときのみを考慮する(平均の二倍以上の人が集まる確率は限りなく0なので0に近似)
    @collecting_deadline.times do |t_people|
      # t_people人来るとき
      when_people_come = simulate_when_people_come(t_people)  # 人がいつ来るか配列で表す EXAMPLE [1, 0, 1] => 時刻0と時刻2に人がくる
      temp_probability_by_poisson = @poisson.probability{ |x| x == t_people }
      method_utility = 0 # 効用 = 精度(正解率) - かかる時間(決定する時刻 / collecting_deadline)
      people_count = 0 # 何人目か
      when_people_come.each_with_index do |elem, index|
        if index >= time_limit && people_count >= 1
          method_utility = (1 - relative_error_by_majority_vote(people_count, possibility_correct)) - WEIGHT * (index.to_f / @collecting_deadline)
          break
        end
        next if elem.nil? # 人が来ない時刻は飛ばす
        people_count += 1
      end
      average_method_utility += temp_probability_by_poisson * method_utility
    end
    average_method_utility
  end

  def half_num(num)
    (num.to_f / 2).ceil
  end

  # nCk 実行例 5C2 = 5 * 4 / 2! = 10
  def combi(n, k)
    k = n - k if 2 * k > n
    return 1 if k == 0
    ((n - k + 1)..n).reduce(&:*) / (1..k).reduce(&:*)
  end

  # 誤差率をもとめる.先にある割合の意見(finish_num)を集めた方を意見集約の結果とするアルゴリズム
  # このアルゴリズムは必要な人数が動的に変化するので、誤差率をfinish_num個の要素にした配列で返す
  def relative_error_array_by_half_opinion(finish_num, possibility_correct) # possibility_correct => 人が正解する確率p
    relative_error = []
    finish_num.times do |t|
      relative_error << ((1 - possibility_correct)**finish_num) * (possibility_correct**t) * combi(finish_num - 1 + t, t)
    end
    relative_error # EXAMPLE (finish_num = 3 )[0.2, 0.3, 0.5] 0.2は3:0で決まるとき 0.3は3:1で決まるとき 0.5は3:2で決まるとき
  end

  # 誤差率をもとめる.people_num人で多数決を行い意見集約の結果とするアルゴリズム
  def relative_error_by_majority_vote(people_num, possibility_correct) # possibility_correct => 人が正解する確率p
    half_num = half_num(people_num)
    relative_error = 0
    half_num.times do |t|
      relative_error += (possibility_correct**t) * ((1 - possibility_correct)**(people_num - t)) * combi(people_num, t)
    end
    relative_error
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method1(possibility_correct) # possibility_correct => 人が正解する確率p
    # 終了する人数を求める
    finish_num = @people_half
    # 誤差率を求め、返す
    relative_error_array_by_half_opinion(finish_num, possibility_correct).inject(:+)
  end

  # 確率pを変動させて、誤差率εの求める
  def baseline_method2(possibility_correct)
    # 多数決を行い誤差率を求め、返す
    relative_error_by_majority_vote(@people_num, possibility_correct)
  end

  # 誤差率εを変動させて、必要な人数Xを求める
  def baseline_method3(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 終了する人数を求める
    temp_finish_num = half_num(people_num)
    # 必要な平均人数
    average_num = 0
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      relative_error_array = relative_error_array_by_half_opinion(temp_finish_num, @possibility_correct)
      expected_error = relative_error_array.inject(:+)
      average_num = 0
      relative_error_array.each_with_index do |e, i| # もっと簡単にかけそう
        average_num += e * (i + temp_finish_num) / expected_error
      end
      break if expected_error <= relative_error
      people_num += 2
      temp_finish_num = half_num(people_num)
    end
    # 必要な人数の平均を返す
    average_num
  end

  # 多数決を意見集約のアルゴリズムとする
  def baseline_method4(relative_error)
    # 必要な人数を求める 1,3,5,,,,と増やす
    people_num = 1
    # 徐々に人数を増やしていき、誤差率を満たす結果の場合ループを終了する
    loop do
      expected_error = relative_error_by_majority_vote(people_num, @possibility_correct)
      break if expected_error <= relative_error
      people_num += 2
    end
    # 必要な人数を返す
    people_num
  end
end

require 'rbplotly'

PEOPELE_NUM = 199
POSSIBILITY_CORRECT = 0.7
LAMBDA_POISSON = 30
DELIMITER = 100

model = Modeling.new(PEOPELE_NUM, POSSIBILITY_CORRECT, LAMBDA_POISSON)

# 縦軸 => 効用utility, 横軸 => 人の正解する確率p
utility_possibility_x_axis = (DELIMITER / 2..DELIMITER).to_a
# 乱数を用いてるので10回分の平均をとる
temp_utility_possibility_y_axis_by_first_person = []
temp_utility_possibility_y_axis_by_majority_vote = []
temp_utility_possibility_y_axis_by_time_limit = []
10.times do
  temp_utility_possibility_y_axis_by_first_person << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_first_person_with_poisson(e.to_f / DELIMITER) }
  temp_utility_possibility_y_axis_by_majority_vote << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_majority_vote_with_poisson(e.to_f / DELIMITER, 5) }
  temp_utility_possibility_y_axis_by_time_limit << utility_possibility_x_axis.map { |e| model.baseline_method_deciding_by_time_limit_with_poisson(e.to_f / DELIMITER, 10) }
end
utility_possibility_y_axis_by_first_person = []
utility_possibility_y_axis_by_majority_vote = []
utility_possibility_y_axis_by_time_limit = []
utility_possibility_x_axis.length.times do |x_axis|
  average_y_first_person = 0
  average_y_majority_vote = 0
  average_y_time_limit = 0
  10.times do |t|
    average_y_first_person +=  temp_utility_possibility_y_axis_by_first_person[t][x_axis] / 10
    average_y_majority_vote +=  temp_utility_possibility_y_axis_by_majority_vote[t][x_axis] / 10
    average_y_time_limit +=  temp_utility_possibility_y_axis_by_time_limit[t][x_axis] / 10
  end
  utility_possibility_y_axis_by_first_person << average_y_first_person
  utility_possibility_y_axis_by_majority_vote << average_y_majority_vote
  utility_possibility_y_axis_by_time_limit << average_y_time_limit
end
utility_possibility_by_first_person_trace = [ {x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_first_person}]
utility_possibility_by_majority_vote_trace = [ {x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_majority_vote}]
utility_possibility_by_time_limit_trace = [ {x: utility_possibility_x_axis, y: utility_possibility_y_axis_by_time_limit}]

pl = Plotly::Plot.new(data: utility_possibility_by_first_person_trace + utility_possibility_by_majority_vote_trace + utility_possibility_by_time_limit_trace)
pl.layout.xaxis = { title: 'possibility_correct' }
pl.layout.yaxis = { title: 'utility' }
pl.show


(pry):513: warning: already initialized constant Modeling::WEIGHT
(pry):263: warning: previous definition of WEIGHT was here
(pry):709: warning: already initialized constant PEOPELE_NUM
(pry):459: warning: previous definition of PEOPELE_NUM was here
(pry):710: warning: already initialized constant POSSIBILITY_CORRECT
(pry):460: warning: previous definition of POSSIBILITY_CORRECT was here
(pry):711: warning: already initialized constant LAMBDA_POISSON
(pry):461: warning: previous definition of LAMBDA_POISSON was here
(pry):712: warning: already initialized constant DELIMITER
(pry):462: warning: previous definition of DELIMITER was here


#<Plotly::Offline::HTML:0x00007fde4d390250 @id="65066577-9cb4-4133-8cfc-80abb9ff0aac", @data=[{:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.4915342182137878, 0.5020565431757672, 0.5101922793737919, 0.5208268772731891, 0.5316945739536069, 0.5403028982725523, 0.5506714055500861, 0.560503693039515, 0.5712426438849463, 0.5818810497929439, 0.5922383916362323, 0.6008567381267171, 0.6115026837038017, 0.6217168353796501, 0.6307080253516788, 0.6426336399968544, 0.6521653515057866, 0.6607739844602982, 0.6719418555287587, 0.6808468806882833, 0.6916739190297777, 0.7008243459967622, 0.711671494192821, 0.7216945123615026, 0.7318491072675246, 0.7412401341425244, 0.7517282846580713, 0.76134355341685, 0.7697514306097778, 0.7823742217045557, 0.7911670388494387, 0.8018174580649664, 0.8121324409473221, 0.8212849316719536, 0.833474608502455, 0.8415952419272706, 0.8506845638389746, 0.8603504939686292, 0.8730026232932616, 0.8808366205535708, 0.890494950544302, 0.9008770833576449, 0.9099767582465013, 0.9220504936329967, 0.9313724961733008, 0.9410970337207147, 0.9515943298511373, 0.9599403810936481, 0.9716990751489717, 0.9808307647495883, 0.9913506895226692]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.42592227749804323, 0.4406039413665599, 0.45940103091522555, 0.47987388336757225, 0.49804061265291505, 0.5189322776704296, 0.5342286178974593, 0.5555195205743351, 0.5721249802222099, 0.5877434889139911, 0.6052544064144684, 0.6239213424063283, 0.6410470264262201, 0.6592648324857813, 0.6723805242864513, 0.6886468764643457, 0.7061104726613948, 0.7210724308797414, 0.7312628872691248, 0.7508643812107, 0.7613159990819371, 0.7746797016166782, 0.7867584334853266, 0.7983018368731736, 0.8083178195366045, 0.8177851835992407, 0.8297789754840239, 0.8360650845126566, 0.8483787993776627, 0.8574448032328581, 0.8651823342818015, 0.8750141880657689, 0.8755816688356384, 0.8866562636993318, 0.89063501706133, 0.8986184464727521, 0.899330381046175, 0.9054162424778243, 0.9075248831788671, 0.9104436739845226, 0.9129967106740141, 0.9199595025094872, 0.9174611594974208, 0.9207127146247794, 0.9227633063370918, 0.9220585295309505, 0.9196352711806843, 0.9231444792703448, 0.9221928254113501, 0.9236934692493944, 0.9249575951807324]}, {:x=>[50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100], :y=>[0.4991730430671715, 0.5342422851457911, 0.5307568891114862, 0.5596220856968595, 0.5628506905335594, 0.5855542224823868, 0.6062120772783955, 0.6264602072152383, 0.6395910628445078, 0.6496975649962891, 0.6695088226553977, 0.680149878583915, 0.6941545223687584, 0.7112811262375028, 0.7264526262898543, 0.7436873413581218, 0.7572802414478944, 0.757981381437656, 0.7715354643364235, 0.7845469582832594, 0.7905742582415205, 0.804062718769901, 0.8074719293554753, 0.8233220561162184, 0.8306086348721883, 0.842887674474523, 0.8488465323895188, 0.8589761662085775, 0.8612967895942929, 0.8676715368992785, 0.8707290444430124, 0.8767139695647803, 0.8815676461417096, 0.8864852890968729, 0.8894260579963283, 0.8950479287881099, 0.8987675537201731, 0.8996097100412468, 0.902419691440295, 0.9061350222114604, 0.9083019280301403, 0.9086821431917582, 0.9104303024174926, 0.9125496422360544, 0.9130538449918589, 0.9142804392865748, 0.9151113457757685, 0.9155656164237247, 0.9162001129674148, 0.9163194697386285, 0.9164309296574757]}], @layout={:xaxis=>{:title=>"possibility_correct"}, :yaxis=>{:title=>"utility"}}, @config={:linkText=>"Export to plot.ly", :showLink=>true}, @embedded=true>

#<CZTop::Socket::PUB:0x7fde4e4e3050 last_endpoint="tcp://127.0.0.1:55497">